In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


# Load Datas

In [ ]:
df = pd.read_csv('sdn_ml.csv')
df.index = df['timestamp']
df.drop('timestamp', axis=1, inplace=True)
df.head()

# Separing dataset

In [ ]:
train_size = int(len(df) * 0.9)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]


# Normalizing

In [ ]:
from sklearn.preprocessing import RobustScaler

In [ ]:
f_columns = ['delay','temperature']

f_transformer = RobustScaler()
lb_transformer = RobustScaler()

f_transformer = f_transformer.fit(train[f_columns].to_numpy())
train.loc[:,f_columns] = f_transformer.transform(train[f_columns].to_numpy())

#lb_transformer = lb_transformer.fit(train[['label']])
#train['label'] = lb_transformer.transform(train[['label']])

In [ ]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i: (i+time_steps),5:8].to_numpy()
        Xs.append(v)
        ys.append(y.iloc[i+time_steps])
    return np.array(Xs), np.array(ys)

In [ ]:
TIME_STEPS = 30

X_train,Y_train = create_dataset(train, train.label, time_steps=TIME_STEPS)
X_test,Y_test = create_dataset(test, test.label, time_steps=TIME_STEPS)

In [ ]:
#deoeting label
# X_train.drop('label', axis=1, inplace=True)
# X_test.drop('label', axis=1, inplace=True)


In [ ]:
X_train

In [ ]:
print(type(X_train), type(Y_train))

# Training

In [ ]:
model = keras.Sequential()
model.add(
    keras.layers.Bidirectional(
        keras.layers.LSTM(
            units=128,
            input_shape=(X_train.shape[1],X_train.shape[2])
        )
    ))
model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Dense(units=128))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.Dense(units=1))
# model.add(tf.keras.layers.ThresholdedReLU(theta=1))

# Compilation

In [ ]:
loss ="mse"
optim = tf.keras.optimizers.SGD(
    learning_rate=0.001)
metrics=["accuracy"]

In [ ]:
model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [ ]:
history = model.fit(
    X_train, Y_train, 
    epochs=50, 
    batch_size= 32,
    validation_split=0.1,
    shuffle=False
)

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='validation')
plt.legend();

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.legend();

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
plt.plot(Y_test, marker='.', label='true')
plt.plot(y_pred,'r',marker='.', label='predicted')
plt.legend();

In [ ]:
from sklearn.metrics import confusion_matrix


In [ ]:
#confusion_matrix(y_test_inv, y_pred_inv)

In [ ]:
y_pred

In [ ]:
Y_test

In [ ]:
# y_train_inv = lb_transformer.inverse_transform(Y_train.reshape(1,-1)) 
# y_test_inv = lb_transformer.inverse_transform(Y_test.reshape(1,-1))
# y_pred_inv = lb_transformer.inverse_transform(y_pred)